In [1]:
print('hi')

hi


In [2]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
google_api_key = os.getenv('GOOGLE_API_KEY')

In [4]:
if google_api_key == "":
    print('Api key not available')
else:
    print('api key found')

api key found


In [5]:
from llama_index.llms.gemini import Gemini
import google.generativeai as genai
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from IPython.display import Markdown, display
from llama_index.core import ServiceContext #i want to keep embedding, model together for the information retrieval 
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.embeddings.gemini import GeminiEmbedding

In [6]:
#configure my api key
genai.configure(api_key=google_api_key)

In [7]:
for models in genai.list_models():
    print(models)

Model(name='models/chat-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 Chat (Legacy)',
      description='A legacy text-only model optimized for chat conversations',
      input_token_limit=4096,
      output_token_limit=1024,
      supported_generation_methods=['generateMessage', 'countMessageTokens'],
      temperature=0.25,
      top_p=0.95,
      top_k=40)
Model(name='models/text-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 (Legacy)',
      description='A legacy model that understands text and generates text as an output',
      input_token_limit=8196,
      output_token_limit=1024,
      supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
      temperature=0.7,
      top_p=0.95,
      top_k=40)
Model(name='models/embedding-gecko-001',
      base_model_id='',
      version='001',
      display_name='Embedding Gecko',
      description='Obtain a distributed representatio

In [9]:
for model in genai.list_models():
    if 'generateContent' in model.supported_generation_methods:
        print(model.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-001
models/gemini-1.5-flash-latest
models/gemini-1.5-pro
models/gemini-1.5-pro-001
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [11]:
documents = SimpleDirectoryReader("../Data")

In [15]:
doc = documents.load_data()

In [16]:
doc

[Document(id_='0e5e2d3b-ba67-4e59-9a25-1a371b529bbe', embedding=None, metadata={'file_path': 'e:\\Amarnath\\GENAI_Basic_To_Advance\\4.QA_System_With_LlamaIndex\u202c_and_GoogleGemini_Using_LlamaIndex_GeminiEmbedding_GeminiPro\\notebook\\..\\Data\\ISRO.txt', 'file_name': 'ISRO.txt', 'file_type': 'text/plain', 'file_size': 11254, 'creation_date': '2024-07-13', 'last_modified_date': '2024-07-13'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='The Indian Space Research Organisation (ISRO[3] /ˈɪsroʊ/)[a] is India\'s national space agency. It operates as the most principal research and development arm of the Department of Space (DoS), which is directly overseen by the Prime Minister of India, with the Chairman of ISRO also serving as the chief executive of t

In [18]:
doc[0].text

'The Indian Space Research Organisation (ISRO[3] /ˈɪsroʊ/)[a] is India\'s national space agency. It operates as the most principal research and development arm of the Department of Space (DoS), which is directly overseen by the Prime Minister of India, with the Chairman of ISRO also serving as the chief executive of the DoS.\r\n\r\nISRO has the world\'s largest constellation of remote-sensing satellites and operates the GAGAN and IRNSS (NavIC) satellite navigation systems. It has sent three missions to the Moon and one to Mars.\r\n\r\nISRO is primarily responsible for space-based operations, space exploration, international space cooperation and the development of related technologies.[4] ISRO is one of the six government space agencies in the world that possess full launch capabilities, the ability to deploy cryogenic engines, the ability to launch extraterrestrial missions and the ability to operate a large fleet of artificial satellites.[5][6][b]ISRO is also one of only four governm

In [51]:
#model
model = Gemini(model='models/text-bison-001',api_key=google_api_key)

In [52]:
#load the embedding model - Gemini
gemini_embed_model =  GeminiEmbedding(model_name="models/embedding-001") 

In [53]:
#configure service context
service_context = ServiceContext.from_defaults(llm=model,embed_model=gemini_embed_model,chunk_size=800,chunk_overlap=20)

C:\Users\Asus-2020\AppData\Local\Temp\ipykernel_13508\2590339349.py:2: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=model,embed_model=gemini_embed_model,chunk_size=800,chunk_overlap=20)


In [54]:
#create index
index = VectorStoreIndex.from_documents(doc,service_context=service_context)

In [55]:
#create local db
index.storage_context.persist()

In [56]:
#Based on the indexing iam going to create an query engine
query_engine = index.as_query_engine()

In [57]:
response = query_engine.query("Abbrevation of ISRO?")
#response = query_engine.query("Who is Paul Graham.")

InvalidArgument: 400 Add an image to use models/gemini-pro-vision, or switch your model to a text model.